In [1]:
#The first and second format provides people with an instantaneous health score.

In [182]:
# Important Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns=None
import datetime
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

# Basic Data combining and eyeballing

In [3]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('test_l0Auv8Q.csv')
fhc=pd.read_csv('First_Health_Camp_Attended.csv')
shc=pd.read_csv('Second_Health_Camp_Attended.csv')
thc=pd.read_csv('Third_Health_Camp_Attended.csv')
patient=pd.read_csv('Patient_Profile.csv')
health_camp=pd.read_csv('Health_Camp_Detail.csv')

In [4]:
df=pd.concat([train,test],ignore_index=True)
df

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5
0,489652,6578,10-Sep-05,4,0,0,0,2
1,507246,6578,18-Aug-05,45,5,0,0,7
2,523729,6534,29-Apr-06,0,0,0,0,0
3,524931,6535,07-Feb-04,0,0,0,0,0
4,521364,6529,28-Feb-06,15,1,0,0,7
...,...,...,...,...,...,...,...,...
110522,507261,6525,13-Apr-06,12,2,0,0,6
110523,513607,6551,03-Nov-06,0,0,0,0,0
110524,498160,6584,17-Jun-06,0,0,0,0,0
110525,502444,6567,13-Jan-07,0,0,0,0,0


In [5]:
df=pd.merge(left=df,right=fhc,on=['Patient_ID','Health_Camp_ID'],how='left')

In [6]:
df=pd.merge(left=df,right=shc,on=['Patient_ID','Health_Camp_ID'],how='left')

In [7]:
df=pd.merge(left=df,right=thc,on=['Patient_ID','Health_Camp_ID'],how='left')

In [8]:
df=pd.merge(left=df,right=health_camp,on=['Health_Camp_ID'],how='left')

In [9]:
df=pd.merge(left=df,right=patient,on=['Patient_ID'],how='left')

In [10]:
# Final combined data
df

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,489652,6578,10-Sep-05,4,0,0,0,2,NaN,NaN,NaN,NaN,2.0,1.0,16-Aug-05,14-Oct-05,Third,G,2,0,0,0,0,None,None,None,06-Dec-04,NaN,NaN
1,507246,6578,18-Aug-05,45,5,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,16-Aug-05,14-Oct-05,Third,G,2,0,0,0,0,1,75,40,08-Sep-04,C,Others
2,523729,6534,29-Apr-06,0,0,0,0,0,NaN,NaN,NaN,0.402054,NaN,NaN,17-Oct-05,07-Nov-07,Second,A,2,0,0,0,0,None,None,None,22-Jun-04,NaN,NaN
3,524931,6535,07-Feb-04,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,01-Feb-04,18-Feb-04,First,E,2,0,0,0,0,None,None,None,07-Feb-04,I,NaN
4,521364,6529,28-Feb-06,15,1,0,0,7,NaN,NaN,NaN,0.845597,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,0,0,0,1,1,70,40,04-Jul-03,I,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,507261,6525,13-Apr-06,12,2,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,08-Apr-06,17-Apr-06,First,C,2,0,0,1,1,1,None,37,17-Feb-04,NaN,NaN
110523,513607,6551,03-Nov-06,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,13-Nov-06,18-Nov-06,Second,D,2,0,0,0,0,None,None,None,14-Sep-05,G,NaN
110524,498160,6584,17-Jun-06,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,04-Aug-06,09-Aug-06,Second,A,2,0,0,0,0,None,None,None,11-Jun-04,A,NaN
110525,502444,6567,13-Jan-07,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,30-Jan-07,04-Feb-07,Second,D,2,0,0,0,0,None,None,None,04-May-05,NaN,NaN


In [11]:
# first 5 rows of the data
df.head()

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category
0,489652,6578,10-Sep-05,4,0,0,0,2,NaN,NaN,NaN,NaN,2.0,1.0,16-Aug-05,14-Oct-05,Third,G,2,0,0,0,0,None,None,None,06-Dec-04,NaN,NaN
1,507246,6578,18-Aug-05,45,5,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,16-Aug-05,14-Oct-05,Third,G,2,0,0,0,0,1,75,40,08-Sep-04,C,Others
2,523729,6534,29-Apr-06,0,0,0,0,0,NaN,NaN,NaN,0.402054,NaN,NaN,17-Oct-05,07-Nov-07,Second,A,2,0,0,0,0,None,None,None,22-Jun-04,NaN,NaN
3,524931,6535,07-Feb-04,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,01-Feb-04,18-Feb-04,First,E,2,0,0,0,0,None,None,None,07-Feb-04,I,NaN
4,521364,6529,28-Feb-06,15,1,0,0,7,NaN,NaN,NaN,0.845597,NaN,NaN,30-Mar-06,03-Apr-06,Second,A,2,0,0,0,1,1,70,40,04-Jul-03,I,Technology


In [12]:
# info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110527 entries, 0 to 110526
Data columns (total 29 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Patient_ID                 110527 non-null  int64  
 1   Health_Camp_ID             110527 non-null  int64  
 2   Registration_Date          110193 non-null  object 
 3   Var1                       110527 non-null  int64  
 4   Var2                       110527 non-null  int64  
 5   Var3                       110527 non-null  int64  
 6   Var4                       110527 non-null  int64  
 7   Var5                       110527 non-null  int64  
 8   Donation                   6218 non-null    float64
 9   Health_Score               6218 non-null    float64
 10  Unnamed: 4                 0 non-null       float64
 11  Health Score               7819 non-null    float64
 12  Number_of_stall_visited    6515 non-null    float64
 13  Last_Stall_Visited_Number  65

In [13]:
# checking for missing values
df.isnull().sum()

Patient_ID                        0
Health_Camp_ID                    0
Registration_Date               334
Var1                              0
Var2                              0
Var3                              0
Var4                              0
Var5                              0
Donation                     104309
Health_Score                 104309
Unnamed: 4                   110527
Health Score                 102708
Number_of_stall_visited      104012
Last_Stall_Visited_Number    104012
Camp_Start_Date                   0
Camp_End_Date                     0
Category1                         0
Category2                         0
Category3                         0
Online_Follower                   0
LinkedIn_Shared                   0
Twitter_Shared                    0
Facebook_Shared                   0
Income                            0
Education_Score                   0
Age                               0
First_Interaction                 0
City_Type                   

In [14]:
# Correlation matrix
df.corr()

,Patient_ID,Health_Camp_ID,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Category3,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared
Patient_ID,1.000000,-0.003258,-0.020146,-0.025879,-0.019222,-0.003562,-0.018712,0.009080,-0.015951,NaN,-0.007679,-0.004544,-0.003966,0.002673,-0.001014,-0.001427,-0.000716,0.013505
Health_Camp_ID,-0.003258,1.000000,0.015970,0.012881,0.005614,0.029103,0.027989,-0.028046,-0.002384,NaN,-0.046576,-0.027669,-0.011031,-0.019883,0.013681,0.013869,0.010366,0.011109
Var1,-0.020146,0.015970,1.000000,0.833019,0.380143,0.376367,0.768390,0.009382,0.021936,NaN,-0.001530,-0.005538,0.004429,-0.010355,0.094739,0.105954,0.037790,0.037798
Var2,-0.025879,0.012881,0.833019,1.000000,0.647705,0.349015,0.709103,0.023718,0.025333,NaN,-0.008197,-0.009197,0.009346,-0.005524,0.076307,0.071804,0.032565,0.028562
Var3,-0.019222,0.005614,0.380143,0.647705,1.000000,0.231215,0.394569,0.012690,0.002420,NaN,-0.004244,-0.003262,0.013572,0.001280,0.013873,0.022728,0.010678,0.013999
Var4,-0.003562,0.029103,0.376367,0.349015,0.231215,1.000000,0.524874,-0.011795,-0.013378,NaN,-0.006043,0.010411,0.019938,-0.000901,0.070847,0.068759,0.065566,0.055998
Var5,-0.018712,0.027989,0.768390,0.709103,0.394569,0.524874,1.000000,0.036438,0.047661,NaN,-0.000872,0.017365,0.022412,-0.023098,0.137912,0.161451,0.105966,0.098312
Donation,0.009080,-0.028046,0.009382,0.023718,0.012690,-0.011795,0.036438,1.000000,0.411888,NaN,NaN,NaN,NaN,0.046535,0.067704,0.052507,0.048912,0.033218
Health_Score,-0.015951,-0.002384,0.021936,0.025333,0.002420,-0.013378,0.047661,0.411888,1.000000,NaN,NaN,NaN,NaN,-0.011796,0.035832,0.011546,0.025264,0.002214
Unnamed: 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Feature Engineering

In [15]:
# Code to covert into datetime
df.Registration_Date=pd.to_datetime(df.Registration_Date,dayfirst=True)

In [16]:
# Creating new features from registrain_date
df['Registraion_day']=df.Registration_Date.dt.day
df['Registraion_month']=df.Registration_Date.dt.month
df['Registraion_year']=df.Registration_Date.dt.year

In [17]:
# Creating new features from camp_start_date and camp_end_date
df.Camp_Start_Date=pd.to_datetime(df.Camp_Start_Date,dayfirst=True)
df.Camp_End_Date=pd.to_datetime(df.Camp_End_Date,dayfirst=True)
df['camp_start_day']=df.Camp_Start_Date.dt.day
df['camp_start_month']=df.Camp_Start_Date.dt.month
df['camp_start_year']=df.Camp_Start_Date.dt.year
df['camp_end_day']=df.Camp_End_Date.dt.day
df['camp_end_month']=df.Camp_End_Date.dt.month
df['camp_end_year']=df.Camp_End_Date.dt.year

In [18]:
# Creating online_presence
df['online_presence']=df.Online_Follower+df.LinkedIn_Shared+df.Twitter_Shared+df.Facebook_Shared

In [19]:
df.drop(columns=['Online_Follower','LinkedIn_Shared','Twitter_Shared','Facebook_Shared'],inplace=True)

In [20]:
# Creating new feature camp duration
df['camp_duration']=(df.Camp_End_Date-df.Camp_Start_Date).dt.days

In [21]:
df.First_Interaction=pd.to_datetime(df.First_Interaction,dayfirst=True)

In [22]:
df['days_interaction_difference']=(df.Registration_Date-df.First_Interaction).dt.days

In [23]:
df

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Registraion_day,Registraion_month,Registraion_year,camp_start_day,camp_start_month,camp_start_year,camp_end_day,camp_end_month,camp_end_year,online_presence,camp_duration,days_interaction_difference
0,489652,6578,2005-09-10,4,0,0,0,2,NaN,NaN,NaN,NaN,2.0,1.0,2005-08-16,2005-10-14,Third,G,2,None,None,None,2004-12-06,NaN,NaN,10.0,9.0,2005.0,16,8,2005,14,10,2005,0,59,278.0
1,507246,6578,2005-08-18,45,5,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-16,2005-10-14,Third,G,2,1,75,40,2004-09-08,C,Others,18.0,8.0,2005.0,16,8,2005,14,10,2005,0,59,344.0
2,523729,6534,2006-04-29,0,0,0,0,0,NaN,NaN,NaN,0.402054,NaN,NaN,2005-10-17,2007-11-07,Second,A,2,None,None,None,2004-06-22,NaN,NaN,29.0,4.0,2006.0,17,10,2005,7,11,2007,0,751,676.0
3,524931,6535,2004-02-07,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-01,2004-02-18,First,E,2,None,None,None,2004-02-07,I,NaN,7.0,2.0,2004.0,1,2,2004,18,2,2004,0,17,0.0
4,521364,6529,2006-02-28,15,1,0,0,7,NaN,NaN,NaN,0.845597,NaN,NaN,2006-03-30,2006-04-03,Second,A,2,1,70,40,2003-07-04,I,Technology,28.0,2.0,2006.0,30,3,2006,3,4,2006,1,4,970.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,507261,6525,2006-04-13,12,2,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,2006-04-08,2006-04-17,First,C,2,1,None,37,2004-02-17,NaN,NaN,13.0,4.0,2006.0,8,4,2006,17,4,2006,2,9,786.0
110523,513607,6551,2006-11-03,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2006-11-13,2006-11-18,Second,D,2,None,None,None,2005-09-14,G,NaN,3.0,11.0,2006.0,13,11,2006,18,11,2006,0,5,415.0
110524,498160,6584,2006-06-17,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2006-08-04,2006-08-09,Second,A,2,None,None,None,2004-06-11,A,NaN,17.0,6.0,2006.0,4,8,2006,9,8,2006,0,5,736.0
110525,502444,6567,2007-01-13,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-30,2007-02-04,Second,D,2,None,None,None,2005-05-04,NaN,NaN,13.0,1.0,2007.0,30,1,2007,4,2,2007,0,5,619.0


In [24]:
# Unique patient per healthcamp and vice versa
df['HC_per_patient']=df.groupby('Patient_ID')['Health_Camp_ID'].transform('nunique')
df['Patient_per_HC']=df.groupby('Health_Camp_ID')['Patient_ID'].transform('nunique')

In [25]:
# Unique patients visited in the bigenning of camp year and month
df['patient_per_year']=df.groupby('camp_start_year')['Patient_ID'].transform('nunique')
df['patient_per_month']=df.groupby('camp_start_month')['Patient_ID'].transform('nunique')

In [27]:
# Unique patients visited in the end of camp year and month
df['patient_per_year_end']=df.groupby('camp_end_year')['Patient_ID'].transform('nunique')
df['patient_per_month_end']=df.groupby('camp_end_month')['Patient_ID'].transform('nunique')

# Target Variable creation

In [30]:
df

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Donation,Health_Score,Unnamed: 4,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number,Camp_Start_Date,Camp_End_Date,Category1,Category2,Category3,Income,Education_Score,Age,First_Interaction,City_Type,Employer_Category,Registraion_day,Registraion_month,Registraion_year,camp_start_day,camp_start_month,camp_start_year,camp_end_day,camp_end_month,camp_end_year,online_presence,camp_duration,days_interaction_difference,HC_per_patient,Patient_per_HC,patient_per_year,patient_per_month,patient_per_year_end,patient_per_month_end
0,489652,6578,2005-09-10,4,0,0,0,2,NaN,NaN,NaN,NaN,2.0,1.0,2005-08-16,2005-10-14,Third,G,2,None,None,None,2004-12-06,NaN,NaN,10.0,9.0,2005.0,16,8,2005,14,10,2005,0,59,278.0,11,2837,22359,10445,17330,6109
1,507246,6578,2005-08-18,45,5,0,0,7,NaN,NaN,NaN,NaN,NaN,NaN,2005-08-16,2005-10-14,Third,G,2,1,75,40,2004-09-08,C,Others,18.0,8.0,2005.0,16,8,2005,14,10,2005,0,59,344.0,26,2837,22359,10445,17330,6109
2,523729,6534,2006-04-29,0,0,0,0,0,NaN,NaN,NaN,0.402054,NaN,NaN,2005-10-17,2007-11-07,Second,A,2,None,None,None,2004-06-22,NaN,NaN,29.0,4.0,2006.0,17,10,2005,7,11,2007,0,751,676.0,7,3597,22359,8242,14379,14643
3,524931,6535,2004-02-07,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-01,2004-02-18,First,E,2,None,None,None,2004-02-07,I,NaN,7.0,2.0,2004.0,1,2,2004,18,2,2004,0,17,0.0,6,1882,10902,8717,6319,11010
4,521364,6529,2006-02-28,15,1,0,0,7,NaN,NaN,NaN,0.845597,NaN,NaN,2006-03-30,2006-04-03,Second,A,2,1,70,40,2003-07-04,I,Technology,28.0,2.0,2006.0,30,3,2006,3,4,2006,1,4,970.0,23,3823,16175,3823,14842,6519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,507261,6525,2006-04-13,12,2,0,0,6,NaN,NaN,NaN,NaN,NaN,NaN,2006-04-08,2006-04-17,First,C,2,1,None,37,2004-02-17,NaN,NaN,13.0,4.0,2006.0,8,4,2006,17,4,2006,2,9,786.0,26,166,16175,6164,14842,6519
110523,513607,6551,2006-11-03,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2006-11-13,2006-11-18,Second,D,2,None,None,None,2005-09-14,G,NaN,3.0,11.0,2006.0,13,11,2006,18,11,2006,0,5,415.0,3,2180,16175,8757,14842,14643
110524,498160,6584,2006-06-17,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2006-08-04,2006-08-09,Second,A,2,None,None,None,2004-06-11,A,NaN,17.0,6.0,2006.0,4,8,2006,9,8,2006,0,5,736.0,16,3041,16175,10445,14842,8663
110525,502444,6567,2007-01-13,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2007-01-30,2007-02-04,Second,D,2,None,None,None,2005-05-04,NaN,NaN,13.0,1.0,2007.0,30,1,2007,4,2,2007,0,5,619.0,5,2441,2579,9076,14379,11010


In [31]:
def outcome(a,b,c,d):
    if((a>0) | (b>0) | (c>0) | (d>0)):
        return 1
    else:
        return 0

In [33]:
df['target']=df.apply(lambda x:outcome(x['Health_Score'],x['Health Score'],x['Number_of_stall_visited'],x['Last_Stall_Visited_Number']),axis=1)

In [34]:
df.target.value_counts()

0    89993
1    20534
Name: target, dtype: int64

In [38]:
# Dropping unnecesary variable
df.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Donation', 'Health_Score', 'Unnamed: 4',
       'Health Score', 'Number_of_stall_visited', 'Last_Stall_Visited_Number',
       'Camp_Start_Date', 'Camp_End_Date', 'Category1', 'Category2',
       'Category3', 'Income', 'Education_Score', 'Age', 'First_Interaction',
       'City_Type', 'Employer_Category', 'Registraion_day',
       'Registraion_month', 'Registraion_year', 'camp_start_day',
       'camp_start_month', 'camp_start_year', 'camp_end_day', 'camp_end_month',
       'camp_end_year', 'online_presence', 'camp_duration',
       'days_interaction_difference', 'HC_per_patient', 'Patient_per_HC',
       'patient_per_year', 'patient_per_month', 'patient_per_year_end',
       'patient_per_month_end', 'target'],
      dtype='object')

In [43]:
keep_columns=[ 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5','Category1', 'Category2',
       'Category3', 'Registraion_day',
       'Registraion_month', 'Registraion_year', 'camp_start_day',
       'camp_start_month', 'camp_start_year', 'camp_end_day', 'camp_end_month',
       'camp_end_year', 'online_presence', 'camp_duration',
       'days_interaction_difference', 'HC_per_patient', 'Patient_per_HC',
       'patient_per_year', 'patient_per_month', 'patient_per_year_end',
       'patient_per_month_end', 'target']

In [44]:
final_df=df.loc[:,keep_columns]

In [45]:
final_df

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Registraion_day,Registraion_month,Registraion_year,camp_start_day,camp_start_month,camp_start_year,camp_end_day,camp_end_month,camp_end_year,online_presence,camp_duration,days_interaction_difference,HC_per_patient,Patient_per_HC,patient_per_year,patient_per_month,patient_per_year_end,patient_per_month_end,target
0,4,0,0,0,2,Third,G,2,10.0,9.0,2005.0,16,8,2005,14,10,2005,0,59,278.0,11,2837,22359,10445,17330,6109,1
1,45,5,0,0,7,Third,G,2,18.0,8.0,2005.0,16,8,2005,14,10,2005,0,59,344.0,26,2837,22359,10445,17330,6109,0
2,0,0,0,0,0,Second,A,2,29.0,4.0,2006.0,17,10,2005,7,11,2007,0,751,676.0,7,3597,22359,8242,14379,14643,1
3,0,0,0,0,0,First,E,2,7.0,2.0,2004.0,1,2,2004,18,2,2004,0,17,0.0,6,1882,10902,8717,6319,11010,0
4,15,1,0,0,7,Second,A,2,28.0,2.0,2006.0,30,3,2006,3,4,2006,1,4,970.0,23,3823,16175,3823,14842,6519,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,12,2,0,0,6,First,C,2,13.0,4.0,2006.0,8,4,2006,17,4,2006,2,9,786.0,26,166,16175,6164,14842,6519,0
110523,0,0,0,0,0,Second,D,2,3.0,11.0,2006.0,13,11,2006,18,11,2006,0,5,415.0,3,2180,16175,8757,14842,14643,0
110524,0,0,0,0,0,Second,A,2,17.0,6.0,2006.0,4,8,2006,9,8,2006,0,5,736.0,16,3041,16175,10445,14842,8663,0
110525,0,0,0,0,0,Second,D,2,13.0,1.0,2007.0,30,1,2007,4,2,2007,0,5,619.0,5,2441,2579,9076,14379,11010,0


In [46]:
del df,fhc,shc,health_camp,thc

In [47]:
final_df

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Registraion_day,Registraion_month,Registraion_year,camp_start_day,camp_start_month,camp_start_year,camp_end_day,camp_end_month,camp_end_year,online_presence,camp_duration,days_interaction_difference,HC_per_patient,Patient_per_HC,patient_per_year,patient_per_month,patient_per_year_end,patient_per_month_end,target
0,4,0,0,0,2,Third,G,2,10.0,9.0,2005.0,16,8,2005,14,10,2005,0,59,278.0,11,2837,22359,10445,17330,6109,1
1,45,5,0,0,7,Third,G,2,18.0,8.0,2005.0,16,8,2005,14,10,2005,0,59,344.0,26,2837,22359,10445,17330,6109,0
2,0,0,0,0,0,Second,A,2,29.0,4.0,2006.0,17,10,2005,7,11,2007,0,751,676.0,7,3597,22359,8242,14379,14643,1
3,0,0,0,0,0,First,E,2,7.0,2.0,2004.0,1,2,2004,18,2,2004,0,17,0.0,6,1882,10902,8717,6319,11010,0
4,15,1,0,0,7,Second,A,2,28.0,2.0,2006.0,30,3,2006,3,4,2006,1,4,970.0,23,3823,16175,3823,14842,6519,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,12,2,0,0,6,First,C,2,13.0,4.0,2006.0,8,4,2006,17,4,2006,2,9,786.0,26,166,16175,6164,14842,6519,0
110523,0,0,0,0,0,Second,D,2,3.0,11.0,2006.0,13,11,2006,18,11,2006,0,5,415.0,3,2180,16175,8757,14842,14643,0
110524,0,0,0,0,0,Second,A,2,17.0,6.0,2006.0,4,8,2006,9,8,2006,0,5,736.0,16,3041,16175,10445,14842,8663,0
110525,0,0,0,0,0,Second,D,2,13.0,1.0,2007.0,30,1,2007,4,2,2007,0,5,619.0,5,2441,2579,9076,14379,11010,0


# Encoding Categorical Columns

In [50]:
final_df.Category1.value_counts()

1    61417
2    27783
3    21327
Name: Category1, dtype: int64

In [49]:
final_df.Category1=final_df.Category1.map({'First':1,'Second':2,'Third':3})

In [51]:
final_df.Category2.value_counts()

E    28488
F    28115
G    21327
A    19041
D     8742
B     2426
C     2388
Name: Category2, dtype: int64

In [56]:
final_df.Category2=pd.factorize(final_df.Category2,sort=True)[0]

# Handling missing values

In [66]:
final_df.isnull().sum()[final_df.isnull().sum()>0]

Registraion_day                334
Registraion_month              334
Registraion_year               334
days_interaction_difference    334
dtype: int64

In [77]:
final_df.Registraion_day.fillna(final_df.Registraion_day.mode()[0],inplace=True)
final_df.Registraion_month.fillna(final_df.Registraion_month.mode()[0],inplace=True)
final_df.Registraion_year.fillna(final_df.Registraion_year.mode()[0],inplace=True)

In [78]:
final_df.days_interaction_difference.fillna(final_df.days_interaction_difference.median(),inplace=True)

In [80]:
final_df.isnull().sum()
# Missing values aar now dealt with

Var1                           0
Var2                           0
Var3                           0
Var4                           0
Var5                           0
Category1                      0
Category2                      0
Category3                      0
Registraion_day                0
Registraion_month              0
Registraion_year               0
camp_start_day                 0
camp_start_month               0
camp_start_year                0
camp_end_day                   0
camp_end_month                 0
camp_end_year                  0
online_presence                0
camp_duration                  0
days_interaction_difference    0
HC_per_patient                 0
Patient_per_HC                 0
patient_per_year               0
patient_per_month              0
patient_per_year_end           0
patient_per_month_end          0
target                         0
dtype: int64

# Splitting of the data

In [82]:
train.shape,test.shape

((75278, 8), (35249, 8))

In [83]:
newtrain=final_df.loc[0:train.shape[0]-1,:]
newtest=final_df.loc[train.shape[0]:,:]

In [86]:
newtest.drop(columns='target',inplace=True)

In [89]:
newtrain.shape,newtest.shape

((75278, 27), (35249, 26))

In [93]:
x_train=newtrain.drop(columns='target')
y_train=newtrain.target
x_test=newtest.copy()

,Var1,Var2,Var3,Var4,Var5,Category1,Category2,Category3,Registraion_day,Registraion_month,Registraion_year,camp_start_day,camp_start_month,camp_start_year,camp_end_day,camp_end_month,camp_end_year,online_presence,camp_duration,days_interaction_difference,HC_per_patient,Patient_per_HC,patient_per_year,patient_per_month,patient_per_year_end,patient_per_month_end,target
0,4,0,0,0,2,3,6,2,10.0,9.0,2005.0,16,8,2005,14,10,2005,0,59,278.0,11,2837,22359,10445,17330,6109,1
1,45,5,0,0,7,3,6,2,18.0,8.0,2005.0,16,8,2005,14,10,2005,0,59,344.0,26,2837,22359,10445,17330,6109,0
2,0,0,0,0,0,2,0,2,29.0,4.0,2006.0,17,10,2005,7,11,2007,0,751,676.0,7,3597,22359,8242,14379,14643,1
3,0,0,0,0,0,1,4,2,7.0,2.0,2004.0,1,2,2004,18,2,2004,0,17,0.0,6,1882,10902,8717,6319,11010,0
4,15,1,0,0,7,2,0,2,28.0,2.0,2006.0,30,3,2006,3,4,2006,1,4,970.0,23,3823,16175,3823,14842,6519,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75273,0,0,0,0,0,1,5,2,3.0,1.0,2005.0,7,8,2004,12,2,2005,0,189,142.0,8,1992,10902,10445,17330,11010,0
75274,0,0,0,0,0,3,6,2,13.0,2.0,2006.0,10,2,2006,25,4,2006,3,74,519.0,33,1744,16175,8717,14842,6519,1
75275,0,0,0,0,0,1,1,2,24.0,5.0,2005.0,19,6,2005,1,7,2005,0,12,13.0,7,2303,22359,8208,17330,7155,1
75276,0,0,0,0,0,1,4,2,22.0,12.0,2004.0,22,12,2004,6,1,2005,0,15,89.0,14,3517,10902,6774,17330,5293,0


# Modelling

## Logistic Regression

In [91]:
lr=LogisticRegression()

In [94]:
lr_model=lr.fit(x_train,y_train)

In [97]:
y_pred=lr_model.predict_proba(x_test)

In [103]:
y_pred_final=y_pred[:,1]

In [104]:
pd.DataFrame(y_pred_final).to_csv('sub.csv')

## Light GBM

In [110]:
# Best parameters yet
lgbm=LGBMClassifier(num_leaves=31,max_depth=10,learning_rate=0.01,n_estimators=500,scale_pos_weight=3)

In [139]:
lgbm=LGBMClassifier(num_leaves=31,max_depth=10,learning_rate=0.01,n_estimators=550,scale_pos_weight=3)

In [140]:
lg_model=lgbm.fit(x_train,y_train)

In [141]:
y_pred=lg_model.predict_proba(x_test)[:,1]

In [142]:
pd.DataFrame(y_pred).to_csv('sub_lg.csv')

## Cross validation

In [170]:
lgbm=LGBMClassifier(num_leaves=31,max_depth=10,learning_rate=0.01,n_estimators=500,scale_pos_weight=3)

In [155]:
dtree=DecisionTreeClassifier()

In [171]:
rfe=RFECV(estimator=lgbm,step=1,min_features_to_select=5,cv=5,verbose=3)

In [172]:
rfe.fit(x_train,y_train)

Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 featur

<bound method SelectorMixin.get_feature_names_out of RFECV(cv=5,
      estimator=LGBMClassifier(learning_rate=0.01, max_depth=10,
                               n_estimators=500, scale_pos_weight=3),
      min_features_to_select=5, verbose=3)>

In [173]:
print(len(list(rfe.get_feature_names_out())))

14


In [174]:
features=rfe.get_feature_names_out()

In [175]:
new_x_train=x_train.loc[:,features]
new_x_test=x_test.loc[:,features]

In [176]:
new_x_train.shape,new_x_test.shape

((75278, 14), (35249, 14))

## Modelling after feature Selection

In [177]:
lgbm=LGBMClassifier(num_leaves=31,max_depth=10,learning_rate=0.01,n_estimators=500,scale_pos_weight=3)

In [178]:
lg_model=lgbm.fit(new_x_train,y_train)

In [179]:
y_pred=lg_model.predict_proba(new_x_test)[:,1]

In [180]:
pd.DataFrame(y_pred).to_csv('sub_lg.csv')

## Cross Validation for LGBM

In [183]:
lgbm=LGBMClassifier(num_leaves=31,max_depth=10,learning_rate=0.01,n_estimators=500,scale_pos_weight=3)

In [189]:
# Best method yet
kfold=KFold(n_splits=5,shuffle=True)
pred_df=pd.DataFrame()
n=5
for i in range(n):
    folds=next(kfold.split(x_train)) # next helps in iterate over the folds
    xtr=x_train.iloc[folds[0]]
    ytr=y_train.iloc[folds[0]]
    lgbm.fit(xtr,ytr)
    pred_df[i]=lgbm.predict_proba(newtest)[:,1]

In [191]:
median_prob=pred_df.median(axis=1)

In [194]:
pd.DataFrame(median_prob).to_csv('cv_sub.csv')

# Summary
* The modelling was driven by the feature engineering we did in this dataframe
* Logistic regression gave us an AUC score of around 0.42
* Lightgbm with parameter tuning gave us an auc score of 0.64088
* fetaure selection did not have much impact on the auc score
* Lightgbm with cross validation and taking median of all probabbilities gave us the best result with an auc score of 0.64545
* We learned that the target variable can be masked in the dataset that can be extracted from the dataset by studying the problem statement meticulosuly